In [ ]:
#Student Name: Éamon OGairbhith

#Student ID:9638296

#Etivity 4. Class built to handle various operations on matrices

In [40]:
class Matrix:
    
    def __init__(self, m):
        self.matrix = m
        self.rows = len(self.matrix)
        self.cols = len(self.matrix[0])
    
    def print_matrix(self):
        for r in range(self.rows):
            for c in range(self.cols):
                print(self.matrix[r][c])

    def size(self):
        """return matrix size (rows, columns)"""
        
        return (self.rows, self.cols)
    
    def perform_operation(self, m2, op):
        """ Perform an operation on this matrix and passed in matrix"""
        
        result = []
        for i, plist in enumerate(self.matrix):
            result.append(tuple(map(op, zip(self.matrix[i], m2[i]))))
        return tuple(result)

    def add(self, m2):
        """ add this matrix to m2 matrix."""
        
        if (self.rows == len(m2)) and (self.cols == len(m2[0])):
            return self.perform_operation(m2, lambda x: x[0]+x[1])
        else:
            return "Matrix dimensions not equal"
    
    def sub(self, m2):
        """ subtract this m2 from this matrix."""

        if (self.rows == len(m2)) and (self.cols == len(m2[0])):
            return self.perform_operation(m2, lambda x: x[0]-x[1])
        else:
            return "Matrix dimensions not equal"

    def mul(self, m2):
        """ multiply this matrix with another
        """
        
        if self.cols == len(m2):
            result = []
            # iterate on row of the first matrix
            for i, plist in enumerate(self.matrix):
                internal_result = []
                for j in range(len(m2[i])):
                    # get the vertical column of the second matrix
                    multiplier = list(zip(*m2))[j]
                    internal_result.append((sum([a*b for a,b in zip(plist,multiplier)])))
                result.append(tuple(internal_result))
            return tuple(result)
        else:
            raise Exception("The Number of Columns in Matrix A must Equal the Number of Rows in Matrix B")
 
    def determinant(self):
        """(r1c1 * r2c2) - (r1c2 * r2c1)
        prerequisite: 2x2 matrix only"""
        
        if self.rows == 2 and self.cols == 2:
            return (self.matrix[0][0] * self.matrix[1][1]) - (self.matrix[0][1] * self.matrix[1][0])
        else:
            raise Exception ("Matrix must be 2x2")
    
    def inverse(self):
        determinant = self.determinant()
        #special case for 2x2 matrix:
        if self.rows == 2 and self.cols == 2:
            return [[self.matrix[1][1]/determinant, -1*self.matrix[0][1]/determinant],
                    [-1*self.matrix[1][0]/determinant, self.matrix[0][0]/determinant]]
        else:
            raise Exception ("Matrix must be 2x2")
    def cross_product(self, v2):
        """ produce the cross product of self and 2nd vector (v2)"""
        if self.cols == 3:
            return [[self.matrix[0][1]*v2[0][2] - self.matrix[0][2]*v2[0][1],
                     self.matrix[0][2]*v2[0][0] - self.matrix[0][0]*v2[0][2], 
                     self.matrix[0][0]*v2[0][1] - self.matrix[0][1]*v2[0][0]]]
        else:
            return "Matrix must be 3 Dimensional Vector"
     
    
    

In [41]:
m1 = [[1,2,3,4],
      [10,20,30,40],
      [-1,200,300,-4],
     [20, 4, 3, 2]]

m1

[[1, 2, 3, 4], [10, 20, 30, 40], [-1, 200, 300, -4], [20, 4, 3, 2]]

In [42]:
m2 = [[10,20,30,40],
      [20, 4, 3, 2],
      [-1,200,300,-4],
     [1,2,3,4]]

m2 

[[10, 20, 30, 40], [20, 4, 3, 2], [-1, 200, 300, -4], [1, 2, 3, 4]]

In [43]:
m1 = Matrix([[1,2,3,4],
      [10,20,30,40],
      [-1,200,300,-4],
     [20, 4, 3, 2]])



In [44]:
m1.add(m2)

((11, 22, 33, 44), (30, 24, 33, 42), (-2, 400, 600, -8), (21, 6, 6, 6))

In [45]:
m1.mul(m2)

((51, 636, 948, 48),
 (510, 6360, 9480, 480),
 (3686, 60772, 90558, -856),
 (279, 1020, 1518, 804))

In [46]:
m3 = [[1], [0], [2], [4]]


In [47]:
m1.mul(m3)

((23,), (230,), (583,), (34,))

In [48]:
m4_2x2 = Matrix([[10, 20], [30, 40]])

In [49]:
m4_2x2.determinant()

-200

In [50]:
m4_2x2.inverse()

[[-0.2, 0.1], [0.15, -0.05]]

In [51]:
v1 = Matrix([[1, 2, 3]])

v2 = [[3, 4, 5]]

v1.cross_product(v2)

[[-2, 4, -2]]